# Projet de Resumé de texte

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install datasets
!pip install sentence_transformers
!pip install nltk sentence_transformers rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=28b52900956e80e135e9d04b5650f

## Importation des packages

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, concatenate_datasets, DatasetDict
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from datasets import Dataset
import pandas as pd
import spacy
import json
import nltk

## Chargement de données

In [ ]:

# Charger le jeu de données initial CNN/Daily Mail
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Combiner toutes les données en une seule
combined_dataset = concatenate_datasets([dataset["train"], dataset["validation"], dataset["test"]])

# Réaliser une nouvelle subdivision en 80/10/10
new_split = combined_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
# Split du 20% obtenu en deux ensembles de validation et de test (chacun représentant 10% de l'ensemble initial)
valid_test_split = new_split["test"].train_test_split(test_size=0.5, shuffle=True, seed=42)

# Création du nouveau dictionnaire de données
new_dataset = DatasetDict({
    "train": new_split["train"],
    "validation": valid_test_split["train"],
    "test": valid_test_split["test"]
})

# Afficher la taille des nouvelles subdivisions
print(new_dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 249576
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 31197
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 31198
    })
})


## Sauvegarde du dataset nettoyé

In [ ]:
# Charger le modèle de langue de SpaCy (par exemple, l'anglais)
nlp = spacy.load("en_core_web_sm")

# Charger et subdiviser l'ensemble de données comme précédemment
dataset = load_dataset("cnn_dailymail", "3.0.0")
combined_dataset = concatenate_datasets([dataset["train"], dataset["validation"], dataset["test"]])
new_split = combined_dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
valid_test_split = new_split["test"].train_test_split(test_size=0.5, shuffle=True, seed=42)
new_dataset = DatasetDict({
    "train": new_split["train"],
    "validation": valid_test_split["train"],
    "test": valid_test_split["test"]
})

# Fonction de nettoyage avec SpaCy
def clean_text(text):
    doc = nlp(text)
    cleaned_tokens = [
        token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct
    ]
    return " ".join(cleaned_tokens)

# Appliquer la fonction de nettoyage à chaque sous-ensemble
for split in ['train', 'validation', 'test']:
    new_dataset[split] = new_dataset[split].map(
        lambda example: {"article": clean_text(example["article"]), "highlights": clean_text(example["highlights"])}
    )

# Sauvegarder chaque sous-ensemble nettoyé dans un fichier JSON
for split in ['train', 'validation', 'test']:
    with open(f"{split}_cleaned.json", "w") as f:
        for example in new_dataset[split]:
            json.dump({"article": example["article"], "highlights": example["highlights"]}, f)
            f.write("\n")

print("Nettoyage et sauvegarde terminés.")


Map:   0%|          | 0/249576 [00:00<?, ? examples/s]

Map:   0%|          | 0/31197 [00:00<?, ? examples/s]

Map:   0%|          | 0/31198 [00:00<?, ? examples/s]

Nettoyage et sauvegarde terminés.


## Chargement des fichiers sauvegardés

In [ ]:
def load_jsonl(file_path):
    """Loads a JSON Lines file into a list of dictionaries."""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:  # Try to parse the line as JSON
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Warning: Skipping invalid JSON line: {line.strip()} - Error: {e}")
                # Handle invalid lines (e.g., skip or attempt to repair)
    return data

# Load data using the function
train_cleaned = pd.DataFrame(load_jsonl('/content/drive/MyDrive/train_cleaned.json'))
test_cleaned = pd.DataFrame(load_jsonl('/content/drive/MyDrive/test_cleaned.json'))
validation_cleaned = pd.DataFrame(load_jsonl('/content/drive/MyDrive/validation_cleaned.json'))

## Modèle de résumé extractif (Utiliser le jeu de données de tests uniquement)

Un résumé extractif sélectionne les phrases les plus pertinentes dans un texte, sans en générer de nouvelles, contrairement au résumé abstrait.\
**Modèle de résumé extractif** en utilisant un encodeur comme BERT

<h3 align="center"> - Approche </h3>

- Utiliser un modèle pré-entraîné **BERT**
- Calculer la **similarité** ou l'importance de chaque phrase
- Classer les phrases selon leur **pertinence** et sélectionner les **phrases clés**.
- Évaluer le résumé généré avec des métriques comme **BLEU** et **ROUGE**.

**Prétraitement des textes**

In [4]:
test_set = dataset['test']

In [5]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")
nltk.download('punkt_tab')

# Fonction pour diviser un texte en phrases
def split_into_sentences(text):
    return sent_tokenize(text)

# Appliquer la division sur l'ensemble de test
test_set = dataset['test']
data_test = test_set.map(lambda x: {"sentences": split_into_sentences(x["article"])})

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

## Extraction des représentations avec BERT

In [6]:
from transformers import BertTokenizer, BertModel
import torch

# Charger le tokenizer et le modèle BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

# Fonction pour encoder les phrases
def encode_sentences(sentences):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    # Représentations des phrases (on prend la première token CLS comme résumé)
    sentence_embeddings = outputs.last_hidden_state[:, 0, :]
    return sentence_embeddings

# Exemple pour un texte
sample_sentences = data_test[0]["sentences"]
sentence_embeddings = encode_sentences(sample_sentences)
print(f"Représentation d'une phrase : {sentence_embeddings[0].shape}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Représentation d'une phrase : torch.Size([768])


## Sélection des phrases clés

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

def extractive_summary(sentences, embeddings, top_k=3):
    # Calculer la moyenne des embeddings (représentation globale du texte)
    document_embedding = torch.mean(embeddings, dim=0).unsqueeze(0)

    # Calculer la similarité cosinus entre chaque phrase et le document
    similarities = cosine_similarity(embeddings.numpy(), document_embedding.numpy())

    # Trier les phrases par pertinence
    ranked_sentences = sorted(zip(similarities, sentences), reverse=True, key=lambda x: x[0])

    # Sélectionner les phrases les plus pertinentes
    summary = " ".join([sentence for _, sentence in ranked_sentences[:top_k]])
    return summary

# Résumé extractif pour un exemple
summary = extractive_summary(sample_sentences, sentence_embeddings)
print(f"Résumé extrait : {summary}")

Résumé extrait : Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court's treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement.



## Évaluation avec **BLEU** et **ROUGE**

In [9]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

# Calculer le score ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
reference = data_test[0]["highlights"]  # Résumé de référence
scores = scorer.score(reference, summary)

# Calculer le score BLEU
reference_bleu = [reference.split()]
candidate_bleu = summary.split()
bleu_score = sentence_bleu(reference_bleu, candidate_bleu)
print(f"\nScores ROUGE : {scores}")
print(f"\nScore BLEU : {bleu_score}\n")


Scores ROUGE : {'rouge1': Score(precision=0.11224489795918367, recall=0.3235294117647059, fmeasure=0.16666666666666669), 'rouge2': Score(precision=0.010309278350515464, recall=0.030303030303030304, fmeasure=0.015384615384615384), 'rougeL': Score(precision=0.08163265306122448, recall=0.23529411764705882, fmeasure=0.12121212121212122)}

Score BLEU : 2.553782876480489e-155




## Résumé complet de l'évaluation sur les 10 premières observations

In [10]:
Data = data_test.select(range(10))

In [11]:
rouge_scores = []
bleu_scores = []

for i, example in enumerate(Data):
    sentences = example["sentences"]
    reference = example["highlights"]

    # Encodage des phrases
    embeddings = encode_sentences(sentences)

    # Résumé extractif
    summary = extractive_summary(sentences, embeddings)

    # Calcul des scores ROUGE
    rouge = scorer.score(reference, summary)
    rouge_scores.append(rouge)

    # Calcul des scores BLEU
    reference_bleu = [reference.split()]
    candidate_bleu = summary.split()
    bleu = sentence_bleu(reference_bleu, candidate_bleu)
    bleu_scores.append(bleu)

# Moyennes des scores
avg_rouge = {key: sum([score[key].fmeasure for score in rouge_scores]) / len(rouge_scores) for key in rouge_scores[0]}
avg_bleu = sum(bleu_scores) / len(bleu_scores)

print(f"Scores moyens :\nROUGE : {avg_rouge}\nBLEU : {avg_bleu}")

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Scores moyens :
ROUGE : {'rouge1': 0.28542866109768317, 'rouge2': 0.0830516468456737, 'rougeL': 0.17956309934313125}
BLEU : 0.016850199064943925


## Modèle de résumé abstrait (Encoder-Decoder)

### Architecture

In [ ]:
# Convertir les dataframes en objets Dataset de Hugging Face
train_dataset = Dataset.from_pandas(train_cleaned)
validation_dataset = Dataset.from_pandas(validation_cleaned)
test_dataset = Dataset.from_pandas(test_cleaned)

# Charger le tokenizer et le modèle
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def preprocess_function(examples):
    # Tokeniser le texte d'entrée et le résumé, ajuster la longueur maximale selon les besoins
    model_inputs = tokenizer(examples['article'], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['highlights'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokeniser les jeux de données
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(preprocess_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/94716 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/31197 [00:00<?, ? examples/s]

### Entraînement du modèle

In [ ]:
# Arguments d'entraînement
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

# Entraîneur
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
)

# Entraîner le modèle
trainer.train()